# Importing the air-routes dataset

In the [tutorial](tutorial.ipynb), we imported the air-routes dataset by using Python under the hood to download a specially-crafted JSON file and feed it to the database.
Here we learn how to achieve the same effect by letting Cozo fetch and import a series of CSV files, without Python's help.

Let's set the database magic up first:

In [1]:
%load_ext pycozo.ipyext_direct

Next, some parameters to make life eaiser (the lines commented out do the same thing by processing local files):

In [2]:
%cozo_set AIR_ROUTES_NODES_URL 'https://raw.githubusercontent.com/cozodb/cozo/dev/cozo-core/tests/air-routes-latest-nodes.csv'
%cozo_set AIR_ROUTES_EDGES_URL 'https://raw.githubusercontent.com/cozodb/cozo/dev/cozo-core/tests/air-routes-latest-edges.csv'
# %cozo_set AIR_ROUTES_NODES_URL 'file://./../../cozo/cozo-core/tests/air-routes-latest-nodes.csv'
# %cozo_set AIR_ROUTES_EDGES_URL 'file://./../../cozo/cozo-core/tests/air-routes-latest-edges.csv'

First, import the `airport` relation:

In [3]:
res[idx, label, typ, code, icao, desc, region, runways, longest, elev, country, city, lat, lon] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any', 'Any', 'Int?', 'Float?', 'Float?', 'Any', 'Any', 'Float?', 'Float?'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)

?[code, icao, desc, region, runways, longest, elev, country, city, lat, lon] :=
    res[idx, label, typ, code, icao, desc, region, runways, longest, elev, country, city, lat, lon],
    label == 'airport'

:replace airport {
    code: String 
    => 
    icao: String, 
    desc: String, 
    region: String, 
    runways: Int, 
    longest: Float, 
    elev: Float, 
    country: String, 
    city: String, 
    lat: Float, 
    lon: Float
}


  × when requesting URL https://raw.githubusercontent.com/cozodb/cozo/dev/cozo-core/tests/air-routes-latest-nodes.csv
  ╰─▶ Connection refused (os error 61)


The `CsvReader` utility downloads a CSV file from the internet and attempts to parse its content into a relation.
When we store the relation, we specified types for the columns. The `code` column acts as a primary key for the `airport` stored relation.

Next is `country`:

In [4]:
res[idx, label, typ, code, icao, desc] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[code, desc] :=
    res[idx, label, typ, code, icao, desc],
    label == 'country'

:replace country {
    code: String
    =>
    desc: String
}

,status
0,OK


`continent`:

In [5]:
res[idx, label, typ, code, icao, desc] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[idx, code, desc] :=
    res[idx, label, typ, code, icao, desc],
    label == 'continent'

:replace continent {
    code: String
    =>
    desc: String
}

,status
0,OK


We need to make a translation table for the indices the data use:

In [6]:
res[idx, label, typ, code] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[idx, code] :=
    res[idx, label, typ, code],

:replace idx2code { idx => code }

,status
0,OK


The `contain` relation contains information on the geographical inclusion of entities:

In [7]:
res[] <~
    CsvReader(types: ['Int', 'Int', 'Int', 'String'],
              url: $AIR_ROUTES_EDGES_URL, 
              has_headers: true)
?[entity, contained] :=
    res[idx, fr_i, to_i, typ],
    typ == 'contains',
    *idx2code[fr_i, entity],
    *idx2code[to_i, contained]


:replace contain { entity: String, contained: String }

,status
0,OK


Finally, the `route`s between the airports. This relation is much larger than the rest and contains about 60k rows:

In [8]:
res[] <~
    CsvReader(types: ['Int', 'Int', 'Int', 'String', 'Float?'],
              url: $AIR_ROUTES_EDGES_URL, 
              has_headers: true)
?[fr, to, dist] :=
    res[idx, fr_i, to_i, typ, dist],
    typ == 'route',
    *idx2code[fr_i, fr],
    *idx2code[to_i, to]

:replace route { fr: String, to: String => dist: Float }

,status
0,OK


We no longer need the `idx2code` relation:

In [9]:
::remove idx2code

,status
0,OK


Let's verify all the relations we want are there:

In [10]:
::relations

,name,arity,access_level,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers
0,airport,11,normal,1,10,0,0,0
1,contain,2,normal,2,0,0,0,0
2,continent,2,normal,1,1,0,0,0
3,country,2,normal,1,1,0,0,0
4,route,3,normal,2,1,0,0,0


Done!